# Cassandra

En **Cassandra** vamos a crear 2 tablas, una donde almacenamos los accidentes en formato desnormalizado, y otra que nos servirá para almacenar el número de accidientes por tipo.

En la tabla que almacena los accidentes vamos a utilizar como **PARTITION KEY** el número de expediente y como **CLUSTERING KEY** la fecha del accidente que nos permitirá ordenar la información tal y como nos interesa para nuestro caso de uso.

Adiccionalmente tendremos un índice en la tabla de accidentes que nos permitirá localizar la información por Distrito

<br><br> 

<img src="images/Cassandra.png" width=600 height=500>

<br><br> 

## Carga de datos en Pandas

In [2]:
import pandas as pd

In [3]:
df = pd.read_excel("data/accidentes.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   num_expediente  614 non-null    object        
 1   fecha           614 non-null    datetime64[ns]
 2   distrito        614 non-null    object        
 3   tipo_accidente  614 non-null    object        
 4   sexo            614 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 24.1+ KB


In [4]:
df.sample(10)

,num_expediente,fecha,distrito,tipo_accidente,sexo
258,2020S008309,2020-06-15,MONCLOA-ARAVACA,Caída,Hombre
55,2020S003108,2020-02-08,HORTALEZA,Caída,Hombre
34,2020S001930,2020-01-26,HORTALEZA,Alcance,Hombre
126,2020S006635,2020-05-07,SAN BLAS-CANILLEJAS,Caída,Hombre
523,2020S013304,2020-09-23,CHAMARTÍN,Choque contra obstáculo fijo,Hombre
147,2020S006934,2020-05-16,VICÁLVARO,Caída,Hombre
3,2020S003622,2020-01-04,RETIRO,Caída,Mujer
285,2020S008582,2020-06-21,MORATALAZ,Caída,Hombre
97,2020S005360,2020-03-07,VICÁLVARO,Colisión frontal,Hombre
335,2020S009494,2020-07-05,MONCLOA-ARAVACA,Caída,Hombre


## Creacción de Estructura en Cassandra

In [6]:
%load_ext cql

In [7]:
%%cql
DROP KEYSPACE accidentes;

'No results.'

In [8]:
%%cql
CREATE KEYSPACE accidentes 
WITH replication = {'class':'SimpleStrategy', 'replication_factor': 1};

'No results.'

In [9]:
%cql USE accidentes;

'No results.'

In [10]:
%%cql 
CREATE TABLE acum_tipo_accidente (
    tipo_accidente          text,
    total         counter,
    PRIMARY KEY (tipo_accidente)
)

'No results.'

In [11]:
%%cql 
CREATE TABLE accidentes (
    num_expediente   text,
    fecha            date,
    distrito         text,
    tipo_accidente   text,
    sexo             text,
    PRIMARY KEY <<TO_BE_COMPLETED>>
)
WITH CLUSTERING ORDER BY (fecha DESC);

'No results.'

In [12]:
%%cql
CREATE INDEX <<TO_BE_COMPLETED>>

'No results.'

## Carga de datos en Cassandra

In [13]:
from cassandra.cluster import Cluster, BatchStatement, ConsistencyLevel
cluster = Cluster()
session = cluster.connect('accidentes')

In [14]:
import dateutil

def insert_accidentes(df):
    
    sql_insert = """
INSERT INTO accidentes (
num_expediente,
fecha,
distrito,
tipo_accidente,
sexo
) VALUES (%s, %s, %s, %s, %s)
"""

    for index in df.index:
        
        data = [
            df["num_expediente"][index],
            (df['fecha'][index]).date(),
            df["distrito"][index],
            df["tipo_accidente"][index],
            df["sexo"][index]
        ]
        
        session.execute(sql_insert, data)             
        session.execute("UPDATE acum_tipo_accidente SET total = total + 1 WHERE tipo_accidente = %s", 
                        [df["tipo_accidente"][index]]
        )

In [15]:
insert_accidentes(df)

## Resolución del caso de uso

### Datos del número de expediente 2020S000044

In [ ]:
%%cql 
<<TO_BE_COMPLETED>>

<img src="images/Cassandra-1.png">

### Accidentes que ocurren en el distrito de Moratalaz ordenados por fecha

In [ ]:
%%cql 
<<TO_BE_COMPLETED>>

<img src="images/Cassandra-2.png">

### Número de accidentes por tipo

In [ ]:
%%cql 
<<TO_BE_COMPLETED>>

<img src="images/Cassandra-3.png">

### Número de accidentes cuyo tipo es colisión frontal

In [ ]:
%%cql
<<TO_BE_COMPLETED>>

<img src="images/Cassandra-4.png">